# Coursera - IBM Data Science Capstone
## Week 3 Project - Segmenting and Clustering Neighborhoods in Toronto
### Part 1 - Building the DataFrame

Import Libraries

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

#Supress default INFO logging

import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

import logging, sys
logging.disable(sys.maxsize)

Download Canadian Neighborhood Data From Wikipedia 

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Toronto_Postal_Code_Data = BeautifulSoup(source, 'lxml')

Parse Content of PostalCode HTML Table and Convert to a Pandas DataFrame

In [3]:
df_toronto = pd.DataFrame(columns = ['Postalcode','Borough','Neighborhood'])

content = Toronto_Postal_Code_Data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

row_count = 0
for tr in table.find_all('tr'):
    if (row_count == 0):
        row_count += 1
        continue
    elem_num = 0
    for td in tr.find_all('td'):
        if elem_num == 0:
            postcode = td.text
            elem_num += 1
        elif elem_num == 1:
            borough = td.text
            elem_num += 1
        elif elem_num == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    df_toronto = df_toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)
    row_count += 1
    

Only process the cells that have an assigned borough. 
Ignore cells with a borough that is Not assigned.

In [4]:
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']
df_toronto.reset_index(drop = True, inplace = True)

If a cell has a borough but a Not assigned neighborhood, 
then the neighborhood will be the same as the borough

In [5]:
idx = 0
for i in range(0,df_toronto.shape[0]):
    if df_toronto.iloc[idx][2] == 'Not assigned':
        df_toronto.iloc[idx][2] = df_toronto.iloc[idx][1]
        idx += 1

More than one neighborhood can exist in one postal code area. 
For example, in the table on the Wikipedia page, you will 
notice that M5A is listed twice and has two 
neighborhoods: Harbourfront and Regent Park. 
These two rows will be combined into one row with 
the neighborhoods separated with a comma as shown in 
row 11 in the above table.

In [6]:
df = df_toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
#display(df)

In [7]:
print(df.shape)
df.to_csv('toronto-data.csv')

(103, 3)
